# Examples using Zeep to Access SOAP Endpoints
- [References](https://docs.python-zeep.org/en/master/index.html)

In [ ]:
import json
import os
from zipfile import ZipFile

from requests import Session
from requests.auth import HTTPBasicAuth
from zeep import Client, Settings
from zeep.cache import SqliteCache
from zeep.transports import Transport


In [ ]:
%%bash

python3 -mzeep ${GENERIC_SERVICE_WSDL}

## Create Soap Client

In [ ]:
erp_uri = os.environ['ERP_SERVICE_WSDL']
user = os.environ['FUSION_USER']
password = os.environ['FUSION_PASSWORD']

session = Session()
session.auth = HTTPBasicAuth(user, password)

settings = Settings(strict=True,
                    xml_huge_tree=True,
                    raw_response=False,
                    force_https=True
                    )
transport = Transport(cache=SqliteCache(), session=session)

client = Client(erp_uri,
                settings=settings,
                transport=transport)

## Initiate Job Submission Workflow

In [ ]:
package = "oracle/apps/ess/custom/Custom/"
job = "BHSTESTUCM"

response = client.service.submitESSJobRequest(jobPackageName=package, jobDefinitionName=job)
response

In [ ]:
status = client.service.getESSJobStatus(requestId=246009)
status

In [ ]:
details = client.service.getESSExecutionDetails(requestId=246009)
json_details = json.loads(details)

json_details

## Handling Attachments
- [Reference](https://www.w3.org/TR/SOAP-attachments/)

In [ ]:
pack = client.service.downloadESSJobExecutionDetails(requestId=246009, fileType="all")

In [ ]:
tmp_folder = 'data'

for a in pack:
    
    print(f"{a.DocumentId}, {a.DocumentName}, {a.DocumentTitle}, {a.ContentType}")
    
    tmp_zip = os.path.join(tmp_folder, a.DocumentName)
    
    with open(tmp_zip, 'wb') as f:
        f.write(a.Content)

    with ZipFile(tmp_zip) as z:
        z.extractall(path=tmp_folder)

## File Download Process

In [ ]:
soap_uri = os.environ['GENERIC_SERVICE_WSDL']

soap_client = Client(soap_uri,
                     settings=settings,
                     transport=transport)

In [ ]:
# ns0:Field(xsd:string, name: xsd:anySimpleType)
# ns0:OptionList(Option: xsd:string[], name: xsd:anySimpleType)
# ns0:ResultSet(Row: ns0:Row[], name: xsd:anySimpleType)
# ns0:Container(Field: ns0:Field[], ResultSet: ns0:ResultSet[], OptionList: ns0:OptionList[], _attr_1: {})
# ns0:Service(
#     User: ns0:Container, 
#     Document: {
#         Field: ns0:Field[], 
#         ResultSet: ns0:ResultSet[], 
#         OptionList: ns0:OptionList[], 
#         File: ns0:File[], 
#         _attr_1: {}}, 
#     IdcService: xsd:anySimpleType)
# GenericSoapOperation(Service: ns0:Service, webKey: xsd:anySimpleType)
#     -> Service: ns0:Service, webKey: xsd:anySimpleType

response = client.service.GenericSoapOperation(Service=, webKey='cs')